In [1]:
from utils import *
import json
import logging
import matplotlib.pyplot as plt

In [2]:
driver = webdriver.Firefox()

# Initialise Google.
driver.get("https://www.google.com/maps")
driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Reject all"]').click()


In [ ]:
with open("data/codes.json", "r") as file:
    code_data = json.load(file)

In [124]:
with open("data/stations.json", "r") as file:
    station_data = json.load(file)

In [5]:
with open("data/nodes.json", "r") as file:
    nodes = json.load(file)

In [101]:
with open("data/overground.npy", "rb") as file:
    og_mat = np.load(file)

In [127]:
with open("data/url.json", "r") as file:
    url_dict = json.load(file)

In [8]:
def get_journey(driver):    
    # Collect stations.
    journey = []
    for entry in driver.find_elements(By.CSS_SELECTOR, 'span[id^=transit_group] h2'):
        # Remove '.' from abbreviations in the station name.
        station: str = re.sub(r"\.", "", entry.text)
        # Remove 'Station' from the station name.
        station = re.sub(r"( Power)? Station", r"", station)
        # Correct for Harrow-on-the-Hill.
        if station == "Harrow on the Hill":
            station = "Harrow-on-the-Hill"
        if len(station) > 0:
            journey.append(station)

    return journey

def get_timestamps(driver):
    # Collect timestamps.
    timestamps = []
    for entry in driver.find_elements(By.CSS_SELECTOR, 'span[id^=transit_group] div:has(~ h2), span[id^=transit_group] > div > div:first-child'):
        timestamp_str: str = re.sub(r"\u202f", "", entry.text)
        if len(timestamp_str) > 0:
            timestamp = datetime.strptime(timestamp_str, "%I:%M%p")
            timestamps.append(timestamp)
    return timestamps

In [125]:
system = "overground"
codes = station_data[system]["codes"]
stations = station_data[system]["stations"]

In [126]:
# Initialise mat.
n = len(stations)
og_mat = np.zeros((n, n), np.float32) 

In [152]:
line = "liberty"
driver.get(url_dict["overground"][line]["upm_rmf"])

# Open journey details.
for element in driver.find_elements(By.CSS_SELECTOR, 'button[aria-label="Toggle details"]'):
    if element.is_displayed():
        element.click()

In [153]:
journey: list[str] = get_journey(driver)
timestamps: list[datetime] = get_timestamps(driver)

# Check extraction.
assert len(journey) == len(timestamps)
assert len(journey) > 0, "No stops found. Check that the url is correct."

In [154]:
journey

['Upminster', 'Emerson Park', 'Romford']

In [ ]:
for station in journey:
    if station not in stations:
        stations.append(station)
        stations.sort()
        codes.insert(stations.index(station), "")
        og_mat = np.insert(og_mat, stations.index(station), 0, axis=0)
        og_mat = np.insert(og_mat, stations.index(station), 0, axis=1)

(81, 81)

In [ ]:
codes[stations.index("")] = ""

In [147]:
line = "lioness"
url = driver.current_url
route = f"{codes[stations.index(journey[0])].lower()}_{codes[stations.index(journey[-1])].lower()}"
print(route)

if line in url_dict[system].keys():
    url_dict[system][line][route] = url
else:
    url_dict[system][line] = { route: url }


wfh_wfj


19

In [155]:
# Calculate travel time between stations.
times = []
for i in range(len(timestamps) - 1):
    time: int = max((timestamps[i+1] - timestamps[i]).seconds//60, 1)
    times.append(time)

In [156]:
beta: float = 0.9

# Update time matrix.
for i in range(len(times)):
    start_ind = stations.index(journey[i])
    end_ind = stations.index(journey[i+1])
    prev_time = og_mat[start_ind, end_ind]
    if prev_time < 1e-16:
        og_mat[start_ind, end_ind] = times[i]
    else:
        og_mat[start_ind, end_ind] = (1 - beta) * times[i] + beta * prev_time
    og_mat[end_ind, start_ind] = og_mat[start_ind, end_ind]
    print(f"{i}. Updated ({start_ind}, {end_ind}) from {prev_time} to {og_mat[start_ind, end_ind]}.")

0. Updated (67, 19) from 0.0 to 4.0.
1. Updated (19, 55) from 0.0 to 5.0.


In [120]:
if line in nodes[system].keys():
    nodes[system][line][route] = journey
else:
    nodes[system][line] = { route: journey }

In [157]:
with open("data/overground.npy", "wb") as file:
    np.save(file, og_mat)

In [122]:
with open("data/nodes.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)

In [151]:
with open("data/url.json", "w") as file:
    data = json.dumps(url_dict, indent=4)
    file.write(data)

In [105]:
with open("data/stations.json", "w") as file:
    data = json.dumps(station_data, indent=4)
    file.write(data)

In [158]:
driver.quit()

In [ ]:
start = "South Kensington"
travel_time: float = 45
stations_ind = []
start_ind: int = stations.index(start)
find_stations_in_range(mat, start_ind, travel_time, stations_ind)


In [ ]:
set(areas[i] for i in stations_ind)

In [ ]:
# node_pos = np.zeros(len(stations))

In [10]:
piccadilly_nodes = np.zeros((len(stations), 3))

In [53]:
piccadilly_nodes[stations.index("Heathrow Terminal 5")]

array([14.,  9.,  2.])

In [26]:
for i in range(len(nodes["piccadilly"]["act_hiv"])):
    piccadilly_nodes[stations.index(nodes["piccadilly"]["act_hiv"][i])] = [14,i,0]

In [15]:
circle_nodes[stations.index("Bayswater")]

array([25.,  0.])

In [16]:
with open("circle.npy", "wb") as file:
    np.save(file, circle_nodes)

In [69]:
nodes = np.hstack((nodes, np.zeros((269,2))))

In [ ]:
nodes = np.zeros((len(stations), 2))

In [90]:
nodes[stations.index("Hammersmith")]

array([7., 0., 0., 0.])

In [91]:
nodes[stations.index("Edgware Road")]

array([10.,  0.,  5.,  0.])

In [92]:
nodes[stations.index("Paddington")]

array([10.,  0.,  4.,  0.])

In [83]:
i = 0
for station in journey:
    print(station)
    nodes[stations.index(station)] = [10,0,i,0]
    i -= 1

Earl's Court
West Brompton
Fulham Broadway
Parsons Green
Putney Bridge
East Putney
Southfields
Wimbledon Park
Wimbledon


In [51]:
with open("piccadilly.npy", "wb") as file:
    np.save(file, piccadilly_nodes)

In [ ]:
driver.quit()

In [28]:
with open("nodes.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)

In [ ]:
with open("data/stations.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)